# Outline
1. [send out jobs](#1)<br>
1. [make sh files](#makesh)<br>
1. [how much longer?](#longer)<br>
1. [redistribute jobs to speed up](#redistribute)<br>
1. [nadult](#nadult)<br>
1. [combine amat files](#combine)<br>
1. [last bit to amat file](#lastbit)<br>
1. [prep for Gibbs sampler](#optimize)<br>
1. [optimize Gibbs sampler](#opt)<br>
1. [combine pedgibbs, us, up](#results)<br>

In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

In [ ]:
599*599 # all poss adult pairwise comps

<a id ='1'></a>
# send out jobs

In [ ]:
# instead of looping through each parent to fill an nadult x nadult matrix, send out each
# ij loop individually. Also need to source previous objects/functions made in 
# 03_parentage_R.ipynb

DIR = '/home/lindb/eckertlab/brandon/teakettle/amat_files'
if not op.exists(DIR):
    os.makedirs(DIR)

for k in np.arange(599): # nadult
    i = k+1 #R doesn't have zeroth elements (lame)
    if i % 100 == True:
        print i
    for w in np.arange(599):
        j = w+1
        text = '''
source('~/eckertlab/brandon/imports.R')

#load dependencies
dname = '~/eckertlab/brandon/teakettle/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        cat(sprintf ("loading %%s file\n",f))
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

#amat <- numeric(0)
#for (k in 1:noff)
#{
#    a <- matrix(0,(nadult+1),(nadult+1)) 
#    amat <- append(amat,list(a))
#}
#instead of making one big amat, just make a list of len==k, where k'th is the ij val
#with large numbers of samples, amat will be big and take a good amount of time to saveRDS()
amat <- rep(0,noff)
    
i <- %s                                                                           #########################
j <- %s                                                                           #########################
cat(sprintf ("i=%%s",i))                                                           
cat(sprintf ("j=%%s",j))                                                           



print ("starting loop")
if(i==j & i<(nadult+1)) #this is the diagonal, the mother and father aren't likely to be the same tree
{
    unimportant = 1+1 # amat is already 0 on the diagonal, save some time by not looping
#         for (k in 1:noff)
#         {
#             amat[k] <- 0   
#         }
}
if(i==j & i==(nadult+1)) #can have two out-of-plot parents
{ 
    nmat  <- numeric(0)
    pg1   <- pgout 
    pg1.b <- pgout 
    pg2   <- pgout 
    pg2.b <- pgout
    print ("starting loci loop")
    for(la in 1:nloci) 
    {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
        #at the la'th locus
        print (la)
        nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) #returns a matrix with ncol = nrow = nallele @ la'th locus
        nmat <- append(nmat,list(nm))
    }
    print ("starting offspring loop")
    for(k in 1:noff)
    {
        print (k)
        pq <- 1 #incorrectly overwriting pq is bad, mkay
        og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the offspring's observed first and second alleles
        og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,]) #get any offspring's observed regenotyping results
        print ("starting loci loop")
        for(la in 1:nloci)
        {
            #print (la)
            #om = probability of observing the given offspring genotype given all possible true genotypes
            om <- omat.func(og,og.b,la) 
            
            #pq = probability of observing the given offspring genotype given the observed parental genotypes
            pq <- pq*sum(nmat[[la]]*om)
            
            #if the posited parents are not the parents, no use in doing futher calculations @ other loci
            if(pq==0) 
            {
                break 
            }
        }
        print ("writing amat")
        amat[k] <- pq #pq same as above, for the k'th offspring for the i'th and j'th parent
        amat[k] <- pq #since i==j, it's ok to flip over diagonal
    }# end offspring loop
}# end two-out-of-plot loop
if(i!=j) #if not on the diagonal
{ #all other combos... 
    nmat <- numeric(0)
    if(i<(nadult+1)) #if posited mother one of those who were sampled
    {
        #gadult# == nrow=nadult ncol=nloci
        pg1 <- rbind(gadult[[1]][i,],gadult[[2]][i,])  #get first and second alleles for all loci
        pg1.b <- rbind(gadult.r[[1]][i,],gadult.r[[2]][i,]) #get first and second alleles for regenotyping
    } 
    if(j<(nadult+1)) #if posited father one of those who were sampled
    {
        pg2 <- rbind(gadultF[[1]][j,],gadultF[[2]][j,])
        pg2.b <- rbind(gadultF.r[[1]][j,],gadultF.r[[2]][j,]) 
    }
    if(i==(nadult+1)) #if posited mother is not one of those who were sampled
    { 
        pg1 <- pgout 
        pg1.b <- pgout
    } 
    if(j==(nadult+1)) #if posited father is not one of those who were sampled
    {
        pg2 <- pgout #put in all zeros
        pg2.b <- pgout #put in all zeros
    }
    print ("starting loci loop")
    for(la in 1:nloci)
    {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
        #at the la'th locus
        print (la)
        nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) 
        nmat <- append(nmat,list(nm))
    }
    print ("starting offspring loop")
    for(k in 1:noff)
    { #pre-calc nmat at all loci for parent pair
        print(k)
        pq <- 1 #incorrectly overwriting is still bad, mkay
        og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the k'th offspring's 1st and 2nd alleles @ all loci
        og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,])
        print ("starting loci loop")
        for(la in 1:nloci)
        {
            print (la)
            #om = probability of observing the given offspring genotype given all possible true genotypes
            om <- omat.func(og,og.b,la)
            #pq = probability of observing the given offspring genotype given the observed parental genotypes
            pq <- pq*sum(nmat[[la]]*om)
            if(pq==0) 
            {
            #if the posited parents are not the parents, no use in doing futher calculations @ other loci
                break 
            }
        }
        amat[k]  <- pq 
        #amat[k] <- pq # I'm pretty sure I don't want to do this, since I'm specifying mother as i 
                             # and father as j
    }# end offspring loop
}#end "all other combos" loop


print ("writing RDS files")
DIR = '~/eckertlab/brandon/teakettle/amat_outfiles/'
amatfname = paste(DIR,'amat_%s_%s.RDS',sep="")                                   ######################### 2x
nmatfname = paste(DIR,'nmat_%s_%s.RDS',sep="")                                   ######################### 2x
cat(sprintf("writing %%s",amatfname))
saveRDS(amat,amatfname)
cat(sprintf("writing %%s",nmatfname))
saveRDS(nmat,nmatfname)


''' % (i,
       j,
       str(i).zfill(3),
       str(j).zfill(3),
       str(i).zfill(3),
       str(j).zfill(3)
          )

        filE = op.join(DIR,"amat_%s_%s.R" % (str(i).zfill(3),str(j).zfill(3)))
        with open(filE,'wb') as o:
            o.write("%s" % text)

In [ ]:
DIR = '/home/lindb/eckertlab/brandon/teakettle/amat_outfiles/'
if not op.exists(DIR):
    os.makedirs(DIR)

# dont forget about amat[[k]] <- amat[[k]]/sum(amat[[k]])
I'm pretty sure I was talking about when I did this the first time without distinguishing 
mother as i and father as j and didn't reflect across diagonal?

I might need to do this once I recombine all of the amats

<a id='makesh'></a>
# make sh files to run R files

In [ ]:
DIR = '/gpfs_fs/home/eckertlab/brandon/teakettle/amat_files/'
rfiles = fs(DIR)
luni(rfiles)

In [ ]:
luni(rfiles) == len(rfiles)

In [ ]:
len(rfiles) == 599*599

In [ ]:
rfiles[0]

In [ ]:
# how many r files per sh file?
# 594 sh files because I'm using 3 other student's accounts to submit jobs muahahahaha
# 3*198 = 594
math.ceil(len(rfiles)/593) # = 606

In [ ]:
# where to put the sh files
DIR = '/home/lindb/eckertlab/brandon/teakettle/amat_runfiles/'
if not op.exists(DIR):
    os.makedirs(DIR)
namedirs = []
for name in sorted(['trevor','mitra','brandon']):
    D = op.join(DIR,name)
    if not op.exists(D):
        mkdir(D)
    namedirs.append(D)
namedirs

In [ ]:
# make sh files: 198 for each namedir, 606 rfiles per sh file
fcount = 0
shcount = 0
rcount = 0
namecount = 0
whichname = 0 #starting out with namedirs[0]
DIR = namedirs[whichname]
for f in sorted(rfiles):
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N make%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (str(shcount).zfill(3),
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount    += 1
    rcount    += 1
    
    if rcount == 606 or fcount == len(rfiles):
        namecount += 1 # only want 198 sh files per person
        if namecount == 198:
            whichname += 1
            DIR = namedirs[whichname]
            namecount = 0
        filE = op.join(DIR,'amat_%s.sh' % str(shcount).zfill(3))
        with open(filE,'wb') as o:
            o.write("%s" % text)
        
        rcount = 0
        shcount += 1

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/amat_runfiles/'
cd(DIR) #so .o files don't end up some place weird
for f in fs(DIR):
    !qsub $f
# ssh into other accounts and submit their jobs too

<a id="longer"></a>
## how much longer is this going to take? (still have to run gibbs sampler after this)

In [ ]:
from datetime import datetime as dt
from datetime import timedelta as tdelta
import calendar
months = calendar.month_name[0:13]

In [ ]:
DIR = '/home/lindb/eckertlab/brandon/teakettle/amat_outfiles/'
files = [f for f in ls(DIR) if 'amat' in f]
l = len(files)
l , l/(600*600), str(dt.now())

In [ ]:
# when will this finish? - total time ~= 21 days
diff = dt.now() - dt(2018,1,5,18,19)
minsdiff = divmod(diff.days * 86400 + diff.seconds, 60)[0]
rate = len(files) / minsdiff
frem = 360000 - len(files)
trem = (frem / rate) / (24*60)
eta = dt.now() + tdelta(days=trem)
print "Estimated to finish on %s %s, %s at %s:%s" % (months[eta.month],eta.day,eta.year,eta.hour,eta.minute)

In [ ]:
# when will this finish? - total time ~= 21 days
diff = dt.now() - dt(2018,1,5,18,19)
minsdiff = divmod(diff.days * 86400 + diff.seconds, 60)[0]
rate = len(files) / minsdiff
frem = 358801 - len(files)
trem = (frem / rate) / (24*60)
eta = dt.now() + tdelta(days=trem)
print "Estimated to finish on %s %s, %s at %s:%s" % (months[eta.month],eta.day,eta.year,eta.hour,eta.minute)

In [ ]:
files[0]

<a id="redistribute"></a>
# redistribute remaining jobs to speed up

In [ ]:
# get a list of the files that should eventually be there
shouldfiles = []
for x in range(599):
    i = str(x+1).zfill(3)
    for y in range(599):
        j = str(y+1).zfill(3)
        filE = '/home/lindb/eckertlab/brandon/teakettle/amat_outfiles/amat_%s_%s.RDS' % (i,j)
        shouldfiles.append(filE)
len(shouldfiles)

In [ ]:
# get a list of files that are there
DIR = '/home/lindb/eckertlab/brandon/teakettle/amat_outfiles/'
files = [op.join(DIR,f) for f in ls(DIR) if 'amat' in f]
len(files)

In [ ]:
# get a list of files that are there
DIR = '/home/lindb/eckertlab/brandon/teakettle/amat_outfiles/'
files = [op.join(DIR,f) for f in ls(DIR) if 'amat' in f]
len(files)

In [ ]:
# figure out which files are missing
rem = set(shouldfiles) - set(files)
rem = sorted([x for x in rem])
len(rem)

In [ ]:
# figure out which files are missing
rem = set(shouldfiles) - set(files)
rem = sorted([x for x in rem])
len(rem)

In [ ]:
op.basename(rem[0]).split('DS')

In [ ]:
rfiles[0],op.dirname(rfiles[0])

In [ ]:
# make a list of remaining R files
rdir = op.dirname(rfiles[0])
remfiles = []
for r in rem:
    b = op.basename(r).split("DS")[0]
    f = op.join(rdir,b)
    remfiles.append(f)
remfiles[0]

In [ ]:
op.exists(remfiles[0])

In [ ]:
namedirs

In [ ]:
remdir = '/home/lindb/eckertlab/brandon/teakettle/amat_runfiles/remaining'
for n in sorted(['brandon','mitra','trevor']):
    DIR = op.join(remdir,n)
    if not op.exists(DIR):
        mkdir(DIR)
remdirs = fs(remdir)
remdirs

In [ ]:
# how many per sh file?
math.ceil(len(remfiles)/593) # = 29

In [ ]:
# make sh files: 198 for each namedir, 606 rfiles per sh file
fcount = 0
shcount = 0
rcount = 0
namecount = 0
whichname = 0 #starting out with namedirs[0]
DIR = remdirs[whichname]
for f in sorted(remfiles):
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N make%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (str(shcount).zfill(3),
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount    += 1
    rcount    += 1
    
    if rcount == 29 or fcount == len(rfiles):
        namecount += 1 # only want 198 sh files per person
        if namecount == 198:
            whichname += 1
            DIR = remdirs[whichname]
            namecount = 0
        filE = op.join(DIR,'amat_%s.sh' % str(shcount).zfill(3))
        with open(filE,'wb') as o:
            o.write("%s" % text)
        
        rcount = 0
        shcount += 1

In [ ]:
# some jobs died for whatever reason, using scripts above figure out which R files remaining ('rem')
# convert rem files to rfiles
DIR = '/home/lindb/eckertlab/brandon/teakettle/amat_files/'
lastfiles = []
for r in rem:
    num = op.basename(r).split("amat_")[1].split(".R")[0]
    f = 'amat_%s.R' % num
    lastfiles.append(op.join(DIR,f))
op.exists(lastfiles[0])

In [ ]:
rem[0]

In [ ]:
lastfiles[0]

In [ ]:
num = str(0).zfill(2)
num

In [ ]:
num = 0
for f in lastfiles:
    text ='''#!/bin/bash
#$ -N lastmake%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd
    
cd %s
R --no-save < %s
''' % (str(num).zfill(2),
       op.dirname(f),
       f)
    filE = '/home/lindb/eckertlab/brandon/teakettle/amat_runfiles/remaining/lastfew/lastfew%s.sh' % str(num).zfill(2)
    with open(filE,'wb') as o:
        o.write("%s" % text)
    num += 1

<a id="nadult"></a>
# make scripts for the nadult+1 cases (i forgot originally)

In [ ]:
rem[0]

In [ ]:
# instead of looping through each parent to fill an nadult x nadult matrix, send out each
# ij loop individually. Also need to source previous objects/functions made in 
# 03_parentage_R.ipynb

DIR = '/home/lindb/eckertlab/brandon/teakettle/amat_files/nadult'
if not op.exists(DIR):
    os.makedirs(DIR)

for k in np.arange(600): # nadult
    i = k+1 #R doesn't have zeroth elements (lame)
    for l in [0,1]:
        if l == 0:
            # i stays the same
            j = 600
        if l == 1: # flip
            j = i
            i = 600
            
        text = '''
source('~/eckertlab/brandon/imports.R')

#load dependencies
dname = '~/eckertlab/brandon/teakettle/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        cat(sprintf ("loading %%s file\n",f))
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

#amat <- numeric(0)
#for (k in 1:noff)
#{
#    a <- matrix(0,(nadult+1),(nadult+1)) 
#    amat <- append(amat,list(a))
#}
#instead of making one big amat, just make a list of len==k, where k'th is the ij val
#with large numbers of samples, amat will be big and take a good amount of time to saveRDS()
amat <- rep(0,noff)
    
i <- %s                                                                           #########################
j <- %s                                                                           #########################
cat(sprintf ("i=%%s",i))                                                           
cat(sprintf ("j=%%s",j))                                                           


pgout <- matrix(0,2,nloci)
print ("starting loop")
if(i==j & i<(nadult+1)) #this is the diagonal, the mother and father aren't likely to be the same tree
{
    unimportant = 1+1 # amat is already 0 on the diagonal, save some time by not looping
#         for (k in 1:noff)
#         {
#             amat[k] <- 0   
#         }
}
if(i==j & i==(nadult+1)) #can have two out-of-plot parents
{ 
    nmat  <- numeric(0)
    pg1   <- pgout 
    pg1.b <- pgout 
    pg2   <- pgout 
    pg2.b <- pgout
    print ("starting loci loop")
    for(la in 1:nloci) 
    {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
        #at the la'th locus
        print (la)
        nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) #returns a matrix with ncol = nrow = nallele @ la'th locus
        nmat <- append(nmat,list(nm))
    }
    print ("starting offspring loop")
    for(k in 1:noff)
    {
        print (k)
        pq <- 1 #incorrectly overwriting pq is bad, mkay
        og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the offspring's observed first and second alleles
        og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,]) #get any offspring's observed regenotyping results
        print ("starting loci loop")
        for(la in 1:nloci)
        {
            #print (la)
            #om = probability of observing the given offspring genotype given all possible true genotypes
            om <- omat.func(og,og.b,la) 
            
            #pq = probability of observing the given offspring genotype given the observed parental genotypes
            pq <- pq*sum(nmat[[la]]*om)
            
            #if the posited parents are not the parents, no use in doing futher calculations @ other loci
            if(pq==0) 
            {
                break 
            }
        }
        print ("writing amat")
        amat[k] <- pq #pq same as above, for the k'th offspring for the i'th and j'th parent
        amat[k] <- pq #since i==j, it's ok to flip over diagonal
    }# end offspring loop
}# end two-out-of-plot loop
if(i!=j) #if not on the diagonal
{ #all other combos... 
    nmat <- numeric(0)
    if(i<(nadult+1)) #if posited mother one of those who were sampled
    {
        #gadult# == nrow=nadult ncol=nloci
        pg1 <- rbind(gadult[[1]][i,],gadult[[2]][i,])  #get first and second alleles for all loci
        pg1.b <- rbind(gadult.r[[1]][i,],gadult.r[[2]][i,]) #get first and second alleles for regenotyping
    } 
    if(j<(nadult+1)) #if posited father one of those who were sampled
    {
        pg2 <- rbind(gadultF[[1]][j,],gadultF[[2]][j,])
        pg2.b <- rbind(gadultF.r[[1]][j,],gadultF.r[[2]][j,]) 
    }
    if(i==(nadult+1)) #if posited mother is not one of those who were sampled
    { 
        pg1 <- pgout 
        pg1.b <- pgout
    } 
    if(j==(nadult+1)) #if posited father is not one of those who were sampled
    {
        pg2 <- pgout #put in all zeros
        pg2.b <- pgout #put in all zeros
    }
    print ("starting loci loop")
    for(la in 1:nloci)
    {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
        #at the la'th locus
        print (la)
        nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) 
        nmat <- append(nmat,list(nm))
    }
    print ("starting offspring loop")
    for(k in 1:noff)
    { #pre-calc nmat at all loci for parent pair
        print(k)
        pq <- 1 #incorrectly overwriting is still bad, mkay
        og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the k'th offspring's 1st and 2nd alleles @ all loci
        og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,])
        print ("starting loci loop")
        for(la in 1:nloci)
        {
            print (la)
            #om = probability of observing the given offspring genotype given all possible true genotypes
            om <- omat.func(og,og.b,la)
            #pq = probability of observing the given offspring genotype given the observed parental genotypes
            pq <- pq*sum(nmat[[la]]*om)
            if(pq==0) 
            {
            #if the posited parents are not the parents, no use in doing futher calculations @ other loci
                break 
            }
        }
        amat[k]  <- pq 
        #amat[k] <- pq # I'm pretty sure I don't want to do this, since I'm specifying mother as i 
                             # and father as j
    }# end offspring loop
}#end "all other combos" loop


print ("writing RDS files")
DIR = '~/eckertlab/brandon/teakettle/amat_outfiles/'
amatfname = paste(DIR,'amat_%s_%s.RDS',sep="")                                   ######################### 2x
nmatfname = paste(DIR,'nmat_%s_%s.RDS',sep="")                                   ######################### 2x
cat(sprintf("writing %%s",amatfname))
saveRDS(amat,amatfname)
cat(sprintf("writing %%s",nmatfname))
saveRDS(nmat,nmatfname)


''' % (i,
       j,
       str(i).zfill(3),
       str(j).zfill(3),
       str(i).zfill(3),
       str(j).zfill(3)
          )

        filE = op.join(DIR,"amat_%s_%s.R" % (str(i).zfill(3),str(j).zfill(3)))
        with open(filE,'wb') as o:
            o.write("%s" % text)

In [ ]:
len([f for f in ls(DIR) if 'sh' not in f]) # N*2 - 1

In [ ]:
rfiles = [f for f in fs(DIR) if not 'sh' in f]
len(rfiles)

In [ ]:
math.ceil(1199/198) # 7.0

In [ ]:
# make sh files: 198 for each namedir, 606 rfiles per sh file
fcount = 0
shcount = 0
rcount = 0
DIR = '/home/lindb/eckertlab/brandon/teakettle/amat_files/nadult/shfiles'
if not op.exists(DIR):
    mkdir(DIR)
for f in sorted(rfiles):
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N make%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (str(shcount).zfill(3),
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount    += 1
    rcount    += 1
    
    if rcount == 7 or fcount == len(rfiles):
        filE = op.join(DIR,'amat_%s.sh' % str(shcount).zfill(3))
        with open(filE,'wb') as o:
            o.write("%s" % text)
        
        rcount = 0
        shcount += 1

In [ ]:
!cd /home/lindb/eckertlab/brandon/teakettle/amat_files/nadult/shfiles

In [ ]:
for f in ls('/home/lindb/eckertlab/brandon/teakettle/amat_files/nadult/shfiles'):
    !qsub $f

<a id='combine'></a>
# combine individual amat RDS files by block, then combine blocks later

In [ ]:
# since there are noff*nadult+1*nadult+1 (274,680,000) entries, it will be faster to create the amat matrix one block
## at a time and then add together later on to create full amat

In [ ]:
# how many ijs per block?
tot = 600*600 #(nadult+1 * nadult+1)
ceiling = math.ceil(tot/198)
ceiling

In [ ]:
# determine the blocks
blocks     = OrderedDict()
blockcount = 0
count      = 0
tcount     = 0
imin       = 1
jmin       = 1
for x in range(600):
    i=x+1
    for y in range(600):
        j=y+1
        count += 1
        tcount += 1
        if count == ceiling or tcount == tot:
            blocks[blockcount] = '%s_%s-%s_%s' % (str(imin).zfill(3),
                                                  str(jmin).zfill(3),
                                                  str(i).zfill(3),
                                                  str(j).zfill(3))
            print blocks[blockcount]
            
            if jmin == 600:
                imin = i + 1
                jmin = 1
                print 'hallelujah'
            else:
                imin = i
                jmin = j + 1
            count = 0
            blockcount += 1     

In [ ]:
len(blocks.keys()) # == number of sh files

In [ ]:
# 043_267-046_285
# 046 - 043 = 3
# 0 for 043
# 1 for 044
# 2 for 045
# 3 for 046 

In [ ]:
# for each block, create its own amat list which will be combined later with others
# write R script for each block command, each get sh file to submit to cluster
rDIR   = '/home/lindb/teakettle/data/moran/amat_blocks/r_files'
rdsDIR = '/home/lindb/teakettle/data/moran/amat_blocks/rds_files'
for d in [rDIR,rdsDIR]:
    if not op.exists(d): # makes amat_blocks and subdir r_files
        mkdir(d)
for b in blocks.keys():
    
    text = '''
# pre-reqs
source('~/imports.R')
amat <- list()
for (k in 1:763){
    amat[[k]] <- matrix(0,600,600)
}
getRDS <- function(i,j){
    filE = sprintf('/home/lindb/eckertlab/brandon/teakettle/amat_outfiles/amat_%%s_%%s.RDS',i,j)
    return(readRDS(filE))
}

# get block info
block = '%s'                                    #####################################################
splits = strsplit(block,'-')[[1]]
imin = strsplit(splits[1],"_")[[1]][1]
jmin = strsplit(splits[1],"_")[[1]][2]
imax = strsplit(splits[2],"_")[[1]][1]
jmax = strsplit(splits[2],"_")[[1]][2]

# get list of files in block, add in ij values to each k'th matrix of the list
j = as.numeric(jmin)
i = as.numeric(imin)
diff = as.numeric(imax)-as.numeric(imin)
count = 0


while (count == 0){ #if we count from jmin to 600
    a <- getRDS(sprintf('%%03d',i),sprintf('%%03d',j))
    for (k in 1:len(a)){
        amat[[k]][i,j] <- a[k]
    }

    # end of while
    j = j+1
    if (j == 601){
        i = i + 1
        count = count + 1
        j = 1
    }
    print(c(i,j))
}
print('end of first block')
while (count < diff){ # just iterate j from 1 to 600
    a <- getRDS(sprintf('%%03d',i),sprintf('%%03d',j))

    for (k in 1:len(a)){
        amat[[k]][i,j] <- a[k]
    }
    
    # end of while
    j = j+1
    if (j==601){
        i = i + 1
        j = 1
        count = count + 1
    }
    print(c(i,j))
}
print('end of second block')
while (count == diff){ # iterating j from 1 to jmax
    a <- getRDS(sprintf('%%03d',i),sprintf('%%03d',j))
    
    for (k in 1:len(a)){
        amat[[k]][i,j] <- a[k]
    }
    # end of while
    j = j+1
    if (j == (as.numeric(jmax) + 1)){
        count = count + 1
    }
    print(c(i,j))
}
print('saving RDS')
filE = fname('%s',sprintf('amat_block_%%s.RDS',block))              #################################
saveRDS(amat,filE)
''' % (blocks[b],
       rdsDIR)
    filE = op.join(rDIR,'block_%s.R' % blocks[b])
    with open(filE,'wb') as o:
        o.write("%s" % text)

In [ ]:
rdsDIR

In [ ]:
rDIR

In [ ]:
# make sh files for the block.R files
DIR = '/home/lindb/teakettle/data/moran/amat_blocks/shfiles/'
if not op.exists(DIR):
    mkdir(DIR)
blockfiles = [f for f in fs(rDIR)]
count = 0
for f in blockfiles:
    text = '''#!/bin/bash
#$ -N block%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (str(count).zfill(3),
       op.dirname(f),
       f)
    count += 1
    filE = op.join(DIR,'amat_block_%s.sh'%str(count).zfill(3))
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
# are RDS files being made?
DIR = rdsDIR
len(ls(DIR))

In [ ]:
cDIR = '/home/lindb/teakettle/data/moran/amat_blocks/combined'
if not op.exists(cDIR):
    mkdir(cDIR)

In [ ]:
# combine the 198 RDS files
text = '''
source('~/imports.R')
DIR = '%s'
files = list.files(DIR,full.names=T)
stopifnot(len(files)==198)

count = 1
for (f in files){
    print (count)
    if (count == 1){
        amat <- readRDS(f)
    }
    else{
        a <- readRDS(f)
        for (k in 1:len(amat)){
            amat[[k]] <- amat[[k]] + a[[k]] # zero + a number = a number (technically a matrix)
        }
    }
    count = count + 1
}
filE = fname('%s','all_blocks_amat.RDS')
saveRDS(amat,filE)
''' % (rdsDIR,
       cDIR)
rfilE = op.join(cDIR,'combine_blocks.R')
with open(rfilE,'w') as o:
    o.write("%s" % text)
    
# write sh file
text = '''#!/bin/bash
#$ -N combine
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (op.dirname(rfilE),rfilE)
filE = op.join(cDIR,'combine_blocks.sh')
with open(filE,'w') as o:
    o.write("%s" % text)

In [ ]:
cDIR

In [ ]:
rdsDIR

<a id='lastbit'></a>
# last bit to the amat 

##### put into R
```R
source('/home/lindb/imports.R')
amat <- readRDS('/home/lindb/teakettle/data/moran/amat_blocks/combined/all_blocks_amat.RDS')

for (k in 1:len(amat)){
    amat[[k]] <- amat[[k]] / sum(amat[[k]])
}

filE = '/home/lindb/teakettle/data/moran/amat_blocks/combined/summed_all_blocks_amat.RDS'
saveRDS(amat,filE)

```

# copy amat to dependencies folder

In [ ]:
src = '/home/lindb/teakettle/data/moran/amat_blocks/combined/summed_all_blocks_amat.RDS'
dst = '/home/lindb/eckertlab/brandon/teakettle/dependencies/summed_all_blocks_amat.RDS'
cp(src,dst)

In [ ]:
op.exists(dst)

<a id='optimize'></a>
# optimize parallelization of Gibbs sampler

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/dependencies/optimize/'
if not op.exists(DIR):
    os.makedirs(DIR)

##### run the following in R to get the stuff inbetween end of amat loop and Gibbs sampler, then write script for sampler

```R
source('/home/lindb/imports.R')

depDIR = '/home/lindb/eckertlab/brandon/teakettle/dependencies'

#load dependencies
dname = depDIR
files = c()
for (f in sort(list.files(dname))){
    if (endsWith(f,'RDS'))
    {
        cat(sprintf ("loading %s file\n",f))
        filE = fname(dname,f)
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(filE))    
    }
}

amat <- summed_all_blocks_amat

## fix pld which is class==character
#pld <- as.numeric(pld)
#saveRDS(pld,'/home/lindb/eckertlab/brandon/teakettle/dependencies/pld.RDS')



###############################################################
################## Step 5: the Gibbs Sampler ##################
###############################################################
#number of Gibbs steps
ncyc <- 20000
saveRDS(ncyc,fname(depDIR,'ncyc.RDS'))
#Starting values for seed and pollen parameters
#usgibbs <- us <- 150
#upgibbs <- up <- 500 
us <- 25
up <- 75
saveRDS(us,fname(depDIR,'us.RDS'))
saveRDS(up,fname(depDIR,'up.RDS'))

###Which parent pairs will you consider? 
##Can consider all with a >0 probability of producing observed 
#offspring genotype (as below) or can set more stringent standard.
ppmat <- numeric(0)
for(k in 1:noff)
{
	print(k)
	b <- matrix(0,(nadult+1),(nadult+1))
	b[which(amat[[k]] > 0)] <- 1 #way faster than this loop below
#	for(i in 1:(nadult+1))
#	{
#		for(j in 1:(nadult+1))
#		{ 
#			##amat_k[i,j] represents the probability of obtaining the observed genotype of 
#			#offspring k given the pedigree, P_k = (i,j), relative to all possible parent combinations
#			if(amat[[k]][i,j]>0)
#			{
#				b[i,j]<-1
#			}
#		}
#	}
	#ppmat is a list of matrices of len(noff), each element a matrix of 0s and 1s. 1s = Amat_k[i,j]>0
	ppmat <- append(ppmat,list(b)) 
}
saveRDS(ppmat,fname(depDIR,'ppmat.RDS'))

#Initial parentage values – start with parents that highest probability of producing observed offspring genotypes.
# my comment: instead of using procedure in Gibbs sampler, I'm choosing the max value since this is what Moran .. 
## had commented in the original text.
Ped.eg <- matrix(0,noff,2)
print("IDing first parents")
for (k in 1:noff)
{ #translates probabilities into a vector, upper triangle first, by column
##################### OLD #####################
#     theta <- c(amat[[k]][upper.tri(amat[[k]])],amat[[k]][lower.tri(amat[[k]])],amat[[k]][(nadult+1),(nadult+1)])
# 	y     <- rmultinom(1,1,theta) 
# 	# ly = twice the total number of parent pair combinations where i!=j (plus one (amat_k[bottom right cell]), minus one)
# 	ly    <- length(y)-1 
# 	ul    <- .5*ly; #total number of parent pair combinations where i!=j
# 	ll    <- ul+1 #total number of parent pair combinations where i!=j plus one
# 	v     <- matrix(0,(nadult+1),(nadult+1)) 
# 	v[upper.tri(v)] <- y[c(1:ul)] #fills in the first half of multinomially distributed random number (parent pair combos)
# 	v[lower.tri(v)] <- y[c(ll:ly)] #fills in the second half of multinomially distributed random number (parent pair combos)
# 	#next add in the multinomially distributed random number corresponding to two parents outside the plot
# 	v[(nadult+1),(nadult+1)] <- y[length(y)] 
# 	Ped.eg[k,1] <- which(apply(v,1,sum)==1) #figure out the i'th coordinate(s) for the parent pair(s) within the matrix
# 	Ped.eg[k,2] <- which(apply(v,2,sum)==1) #figure out the j'th coordinate(s) for the parent pair(s) within the matrix
##################### /OLD #####################
##################### new #####################
    # start with the parents with highest probability, or a random pair among ties
    print(k)
    coords <- which(amat[[k]]==max(amat[[k]]),arr.ind=T)
    if (nrow(coords)==1)
    {
        Ped.eg[k,1] <- coords[1]
        Ped.eg[k,2] <- coords[2]
    }
    else {
        probs <- runif(nrow(coords))
        choice = which(probs==max(probs))
        coords = as.vector(coords[choice,])
        Ped.eg[k,1] <- coords[1]
        Ped.eg[k,2] <- coords[2]
    }
##################### /new #####################
}

saveRDS(Ped.eg,fname(depDIR,'Ped.eg.RDS'))


## to hold parentage estimates
#ped.gibbs <- numeric(0)
#ped.gibbs <- append(ped.gibbs,list(matrix(0,noff,2)))
#ped.gibbs[[1]] <- Ped.eg
## new
#do this, but save a little time: ped.gibbs <- Ped.eg
# saveRDS(list(Ped.eg),fname(depDIR,'ped.gibbs.RDS')) # dont want this bc i'm not using list in gibbs
saveRDS(Ped.eg,fname(depDIR,'ped.gibbs.RDS'))

#special function to speed up gibbs sampler
dostuff <- function(k){
	#ldf and ldm are seq of hypothetical father/mother distances from plot
	#pol.pr is nrow=ncol=nadult, where rows are the recipients of pollen, cols the donor, elements the probability
	#sd.pr is nrow=noffspring ncol=nadult, element is probability of seed dispersal to offspring from adult
	#nadult = 558, noff=718
	sp.pr   <- pol.pr  *matrix(sd.pr[k,],nadult+ldm,nadult+ldf)        #dim(sp.pr) == nadult+10 x nadult + 10
	sp.pr.s <- pol.pr.s*matrix(sd.pr.s[k,],nadult+ldm,nadult+ldf)    #dim(sp.pr.s) == nadult+10 x nadult + 10

	sp.pr   <- sp.pr/sum(sp.pr) 
	sp.pr.s <- sp.pr.s/sum(sp.pr.s)
	if(A1[k]<nadult+1) # if mother in stand
	{ 
	  if(A2[k]<nadult+1) #if both in stand
	  {
	    p     <- sp.pr[A1[k],A2[k]]
	    pstar <- sp.pr.s[A1[k],A2[k]] 
	  }
	  if(A2[k]>nadult) #if father outside stand
	  { 
	    p     <- sum(sp.pr[A1[k],(nadult+1):(nadult+ldf)]) 
	    pstar <- sum(sp.pr.s[A1[k],(nadult+1):(nadult+ldf)])
	  } 
	}
	if(A1[k]>nadult) #if mother outside stand
	{  
	  if(A2[k]<nadult+1) #if father in stand
	  {
	    p     <- sum(sp.pr[(nadult+1):(nadult+ldm),A2[k]])
	    pstar <- sum(sp.pr.s[(nadult+1):(nadult+ldm),A2[k]]) 
	  }
	  if(A2[k]>nadult) #if both outside the stand
	  { 
	    p     <- sum(sp.pr[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)]) 
	    pstar <- sum(sp.pr.s[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)])
	  } 
	}
	p[which(p == 0)] <- 1e-323
	pstar[which(pstar == 0)] <- 1e-323
	#"pre-propose" parents for next loop of gibbs sampler
	#translates probabilities into a vector, upper triangle first, by column 
	theta <- c(ppmat[[k]][upper.tri(ppmat[[k]])],ppmat[[k]][lower.tri(ppmat[[k]])],ppmat[[k]][(nadult+ 1),(nadult+1)])
	y     <- rmultinom(1,1,theta) 
	ly    <- length(y)-1
	ul    <- .5*ly; 
	ll    <- ul+1
	v     <- matrix(0,(nadult+1),(nadult+1)) 
	v[upper.tri(v)]<- y[c(1:ul)]
	v[lower.tri(v)] <- y[c(ll:ly)] 
	v[(nadult+1),(nadult+1)] <- y[length(y)]
	# ped.new[k,1] <- which(apply(v,1,sum)==1) 
	# ped.new[k,2] <- which(apply(v,2,sum)==1)
	ped.new1 <- which(apply(v,1,sum)==1) 
	ped.new2 <- which(apply(v,2,sum)==1)
	tnow   <- p
	tnow.s <- pstar
	if(ped.new1<nadult+1)
	{ 
	  if(ped.new2<nadult+1)
	  {
	    tnew   <- sp.pr[ped.new1,ped.new2]
	    tnew.s <- sp.pr.s[ped.new1,ped.new2]
	  }
	  if(ped.new2>nadult)
	  { 
	    tnew   <- sum(sp.pr[ped.new1,(nadult+1):(nadult+ldf)]) 
	    tnew.s <- sum(sp.pr.s[ped.new1,(nadult+1):(nadult+ldf)])
	  } 
	}
	if(ped.new1>nadult)
	{ 
	  if(ped.new2<nadult+1)
	  {
	    tnew   <- sum(sp.pr[(nadult+1):(nadult+ldm),ped.new2])
	    tnew.s <- sum(sp.pr.s[(nadult+1):(nadult+ldm),ped.new2])
	  }
	  if(ped.new2>nadult)
	  { 
	    tnew   <- sum(sp.pr[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)]) 
	    tnew.s <- sum(sp.pr.s[(nadult+1):(nadult+ldm),(nadult+1):(nadult+ldf)])
	  } 
	}
	return(list(k,
	p,
	pstar,
	ped.new1,
	ped.new2,
	tnow,
	tnow.s,
	tnew,
	tnew.s
	))
}
saveRDS(dostuff,fname(depDIR,'dostuff.RDS'))

```

<a id='opt'></a>
# Optimize Gibbs sampler

In [ ]:
engines = ['04',
           '08',
           '12',
           '16',
           '20',
           '24',
           '30',
           '35',
           '40',
           '45',
           '50',
           '56']
for e in engines:    
    text = '''
source("/home/lindb/imports.R")

eng <- as.numeric(%s)

#load dependencies
dname = "/home/lindb/eckertlab/brandon/teakettle/dependencies"
files = c()
for (f in sort(list.files(dname))){
    if (endsWith(f,'RDS'))
    {
        cat(sprintf ("loading %%s file\n",f))
        filE = fname(dname,f)
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(filE))    
    }
}

amat <- summed_all_blocks_amat

library(snow)
cl = makeSOCKcluster(eng)
exports = c("ppmat",
            "ldf",
            "ldm",
            "nadult"
            )
clusterExport(cl,exports)

ncyc <- 5
loop <- as.array(1:noff)
#Begin Gibbs Sampler:
#set all NA probs to ~0 -
cat(print("starting Gibbs\n"))
for(d in 1:ncyc)
{
    start <- Sys.time()
    print(d)

    ped.gibbsnext <- matrix(0,noff,2)
    ped.new   <- matrix(0,noff,2)
    tnow      <- rep(0,noff)
    tnow.s    <- rep(0,noff) 
    tnew      <- rep(0,noff)
    tnew.s    <- rep(0,noff)
    if(d==1)
    { #just the first time
        #Probability of seed dispersal from sampled adult to location of each seedling:
        dprob <- dprob.func(us) #seedling shape parameter, us, defined around line 1042
        dprob[which(is.na(dprob))] <- 1e-100 #these are because I accidentally included 
        #4 trees without coordinates, no going back until after Evoln
        
        #Probability of pollen dispersal between sampled adults:
        pprob <- pprob.func(up) #pollen shape parameter, up, defined around line 1043
        pprob[which(is.na(pprob))] <- 1e-100 
        
        #Expected seed arriving at seedling plots from outside plot:
        ##osd[i,j] is the total probability of a seed reaching the ith seedlings location 
        #from an unsampled tree j 
        ##distance units outside of the plot
        osd   <- out.sd.func(us)
        osd[which(is.na(osd))] <- 1e-100
        
        #Pollen expected to go from sampled adults to hypothetical outside adults:
        ##iopol[i,j] is the total probability of pollen from the ith sampled tree's
        ##location reaching the jth distance unit outside of the plot
        iopol <- iopol.func(up)
        iopol[which(is.na(iopol))] <- 1e-100
        
        #Expected pollen arriving at sampled trees from outside:
        op    <- out.pol.func(up) 
        op[which(is.na(op))] <- 1e-100
        
        #Pollen expected to go from hypothetical outer tree to outer tree:
        otop  <- otop.func(up)
        otop[which(is.na(otop))] <- 1e-100
    } # end "if first loop"
    
    #### propose new dispersal parameters
    #current parent estimate 
    #A1 <- ped.gibbs[[d]][,1] # proposed mothers of offspring k (see noff loop below)
    #A2 <- ped.gibbs[[d]][,2] # proposed fathers of offspring k (see noff loop below)
    #faster current parent  estimate
    A1 <- ped.gibbs[,1] # proposed mothers of offspring k (see noff loop below)
    A2 <- ped.gibbs[,2] # proposed fathers of offspring k (see noff loop below)
    usstar   <- tnorm(1,los,his,us,20) #pick a new seed dispersal parameter 
    upstar   <- tnorm(1,lop,hip,up,20) #pick a new pollen dispersal parameter
    dprob.s  <- dprob.func(usstar)     #seed dispersal probs
    dprob.s[which(is.na(dprob.s))] <- 1e-100
    pprob.s  <- pprob.func(upstar)     #pollen dispersal probs
    pprob.s[which(is.na(pprob.s))] <- 1e-100
    osd.s    <- out.sd.func(usstar)    #seed to plot [row] from outside mother[col]
    osd.s[which(is.na(osd.s))]     <- 1e-100
    iopol.s  <- iopol.func(upstar)     #pollen to outside mother[col] from inside father[row] 
    iopol.s[which(is.na(iopol.s))] <- 1e-100
    op.s     <- out.pol.func(upstar)   #pollen to inside mother [row] from outside fathers [col] 
    op.s[which(is.na(op.s))]       <- 1e-100
    otop.s   <- otop.func(upstar)      #pollen to outside mother[row] from outside fathers[col]
    otop.s[which(is.na(otop.s))]   <- 1e-100
    #pollen recieved by out mother[i] from any out father:
    otopsum  <- apply(otop,1,sum)
    otopsum.s<- apply(otop.s,1,sum)
    # p        <- rep(NA,noff) 
    # pstar    <- rep(NA,noff)
    #rows = recipients, col= donors
    wmat     <- matrix(w,noff,nadult,byrow=T) # w is the weights vector
    polmat   <- matrix(w,nadult,nadult,byrow=T)
    #pld is the offspring plot size, in my case == 1m^2
    plmat    <- matrix(pld,noff,nadult) #gives a matrix where each column repeats the area of 
    #the offsprings plot sizes
    sd.pr    <- dprob*wmat*plmat 		#prob of seed dispersal from sampled adult to sampled seedling 
    sd.pr.s  <- dprob.s*wmat*plmat
    sd.pr    <- cbind(sd.pr,osd) 		#seed received from inside adults + outside adults 
    sd.pr.s  <- cbind(sd.pr.s,osd.s)
    pol.pr   <- pprob*polmat 
    pol.pr.s <- pprob.s*polmat
    pol.pr   <- cbind(pol.pr,op) 		#pollen received by adults from inside + outside adults 
    pol.pr.s <- cbind(pol.pr.s,op.s) 	#pollen received by outside adults from inside + outside adults 
    om       <- cbind(t(iopol),otop); 
    om.s     <- cbind(t(iopol.s),otop.s)
    pol.pr   <- rbind(pol.pr,om) 
    pol.pr.s <- rbind(pol.pr.s,om.s) 

    ### speed up the old for-loop
    exports = c("A1",
                "A2",
                "pol.pr",
                "pol.pr.s",
                "sd.pr",
                "sd.pr.s")
    print('exporting exports')
    clusterExport(cl,exports)
    out <- parApply(cl,loop,1,dostuff)
    p <- pstar <- tnow <- tnow.s <- tnew <- tnew.s <- pednew1 <- pednew2 <- rep(NA,noff)
    for(k in 1:noff){
        p[k]        <- out[[k]][[2]]
        pstar[k]    <- out[[k]][[3]]
        pednew1[k]	<- out[[k]][[4]]
        pednew2[k]	<- out[[k]][[5]]
        tnow[k]		<- out[[k]][[6]]
        tnow.s[k]	<- out[[k]][[7]]
        tnew[k] 	<- out[[k]][[8]]
        tnew.s[k]	<- out[[k]][[9]]
    }
    ped.new <- cbind(pednew1,pednew2)
    print('export complete')
    ### end speed up of old loop


    q1 <- sum(log(p))
    q2 <- sum(log(pstar))

    #sum over all offspring, times priors
    pnow <- q1+dnorm(up,mpp,sdpp,log=TRUE)+dnorm(us,msp,sdsp,log=TRUE)
    pnew <- q2+dnorm(upstar,mpp,sdpp,log=TRUE)+dnorm(usstar,msp,sdsp,log=TRUE)
    r    <- exp(pnew-pnow)
    z    <- runif(1,0,1) #accept or reject proposed values 
    if(z < r)
    {
        up    <- upstar 
        us    <- usstar 
        dprob <- dprob.s
        pprob <- pprob.s 
        osd   <- osd.s 
        iopol <- iopol.s 
        op    <- op.s
        otop  <- otop.s
        tnew  <- tnew.s
        tnow  <- tnow.s
    }
    x <- d
    while (nchar(x)<5)
    {
        x = sprintf("0%%s",x)
    }
    ###usgibbs <- c(usgibbs,us) #### save
    #saveRDS(us,sprintf("%%s/dependencies/usgibbs_%%s",DIR,x))
    ###upgibbs <- c(upgibbs,up) #### save
    #saveRDS(up,sprintf("%%s/dependencies/upgibbs_%%s",DIR,x))

    ####propose pedigree 
    print('proposing pedigree')
    for(k in 1:noff) #super fast, not worth multiprocessing
    {
        #cat(sprintf("pedigree %%s\\n",k))
        #keep track of parameter values
        #probability that current or proposed parents could produce observed offspring genotype 
        #gnow <- amat[[k]][ped.gibbs[[d]][k,1],ped.gibbs[[d]][k,2]]
        #gnew <- amat[[k]][ped.new[k,1],ped.new[k,2]]
        #faster probability that current or proposed parents could produce observed offspring genotype 
        gnow <- amat[[k]][ped.gibbs[k,1],ped.gibbs[k,2]]
        gnew <- amat[[k]][ped.new[k,1],ped.new[k,2]]

        #...combined with probability of dispersal given current dispersal parameters 
        pnow <- log(gnow)+log(tnow[k])
        pnew <- log(gnew)+log(tnew[k])
        r <- exp(pnew-pnow)
        z <- runif(1,0,1)
        if(z < r)
        {#accept or reject 
            #ped.gibbs[[d+1]][k,1]<-ped.new[k,1] 
            #ped.gibbs[[d+1]][k,2]<-ped.new[k,2]
            #faster accept or reject
            ped.gibbsnext[k,1]<-ped.new[k,1] 
            ped.gibbsnext[k,2]<-ped.new[k,2]
        }
        if(z >= r)
        {
            #ped.gibbs[[d+1]][k,1]<-ped.gibbs[[d]][k,1]
            #ped.gibbs[[d+1]][k,2]<- ped.gibbs[[d]][k,2] 
            #faster
            ped.gibbsnext[k,1]<-ped.gibbs[k,1]
            ped.gibbsnext[k,2]<- ped.gibbs[k,2] 
        }
    }# end k loop over offspring 
    x <- d
    while (nchar(x)<5)
    {
        x = sprintf("0%%s",x)
    }
    #saveRDS(ped.gibbs,sprintf("/home/lindb/teakettle/data/gibbs/cycle_data/ped.gibbs_%%s.RDS",x))
    ped.gibbs <- ped.gibbsnext
    End <- Sys.time()
    print(d)
    print(difftime(End,start))
    
    cyinfo = difftime(End,start)
    filE = sprintf('/home/lindb/teakettle/data/gibbs/engines/rds_files/eng-%%s_cyc-%%s.RDS',eng,d)
    saveRDS(cyinfo,filE)
}#end cycle
stopCluster(cl)

''' % (e)
    DIR  = '/home/lindb/teakettle/data/gibbs/engines'
    iDIR = '/home/lindb/teakettle/data/gibbs/engines/rds_files'
    if not op.exists(iDIR):
        mkdir(iDIR)
    filE = op.join(DIR,'gibbs_%s.R' % str(e).zfill(3))
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
DIR = '/home/lindb/teakettle/data/gibbs/engines'
files = sorted([f for f in fs(DIR) if '.R' in f and '004' not in f]) # i ran 004 to make sure it would work
for f in files:
    print f
count = 0
for f in files:
    if count == 0:
        text = '''#!/bin/bash
#$ -N engtest
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd
#$ -pe smp_2 56

cd %s
R --no-save < %s
''' % (op.dirname(f),
       f
      )
    else:
        newtext ='''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f
      )
        text = text + newtext
    count += 1

filE = op.join(DIR,'engine_test.sh')
with open(filE,'w') as o:
    o.write("%s" % text)

In [ ]:
engines

#### put into R to figure out winner
```R
source('/home/lindb/imports.R')

files = list.files('/home/lindb/teakettle/data/gibbs/engines/rds_files',full.name=T)

res = list()
for (n in c(4,8,12,16,10,24,30,50,56)){
    res[[n]] <- c()
    names(res)[n] <- as.character(n)
}
for (f in sort(files)){
    bname = basename(f)
    splits = strsplit(bname,"_")[[1]]
    eng = strsplit(splits[1],"-")[[1]][2]
    cyc = strsplit(strsplit(splits[2],"-")[[1]][2],".RDS")[[1]]
    r = readRDS(f)
    print(f)
    print(r)
    if (cyc !=1){ # making sure the d==1 doesn't throw off
        res[[eng]] <- c(res[[eng]],r)
    }
    # res[[eng]] <- c(res[[eng]],r)
}

for (r in names(res)){
    m = mean(res[[r]])
    print (r)
    print (m)
}

```

In [ ]:
# 35 engines was the winner @ ~ 13sec per cycle

In [ ]:
# write the Gibbs sampler for 35 engines, save stuff from each cycle along the way
## by saving along the way, don't risk memory issues by building a list (ped.gibbs from original script)
## of length == ncyc 
DIR = '/home/lindb/teakettle/data/gibbs/first/cycfiles'
if not op.exists(DIR):
    mkdir(DIR)
    
text = '''
source("/home/lindb/imports.R")

eng <- as.numeric(35)

#load dependencies
dname = "/home/lindb/eckertlab/brandon/teakettle/dependencies"
files = c()
for (f in sort(list.files(dname))){
    if (endsWith(f,'RDS'))
    {
        cat(sprintf ("loading %s file\n",f))
        filE = fname(dname,f)
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(filE))    
    }
}

amat <- summed_all_blocks_amat

cat(sprintf('making cluster\\n'))
library(snow)
cl = makeSOCKcluster(eng)
cat(print('exporting first exports\\n'))
exports = c("ppmat",
            "ldf",
            "ldm",
            "nadult"
            )
clusterExport(cl,exports)

outdir = '/home/lindb/teakettle/data/gibbs/first/cycfiles'

#ncyc <- 5
loop <- as.array(1:noff)
#Begin Gibbs Sampler:
#set all NA probs to ~0 -
cat(print("starting Gibbs\\n"))
for(d in 1:ncyc)
{
#    start <- Sys.time()
    print(d)

    ped.gibbsnext <- matrix(0,noff,2)
    ped.new   <- matrix(0,noff,2)
    tnow      <- rep(0,noff)
    tnow.s    <- rep(0,noff) 
    tnew      <- rep(0,noff)
    tnew.s    <- rep(0,noff)
    if(d==1)
    { #just the first time
        #Probability of seed dispersal from sampled adult to location of each seedling:
        dprob <- dprob.func(us) #seedling shape parameter, us, defined around line 1042
        dprob[which(is.na(dprob))] <- 1e-100 #these are because I accidentally included 
        #4 trees without coordinates, no going back until after Evoln
        
        #Probability of pollen dispersal between sampled adults:
        pprob <- pprob.func(up) #pollen shape parameter, up, defined around line 1043
        pprob[which(is.na(pprob))] <- 1e-100 
        
        #Expected seed arriving at seedling plots from outside plot:
        ##osd[i,j] is the total probability of a seed reaching the ith seedlings location 
        #from an unsampled tree j 
        ##distance units outside of the plot
        osd   <- out.sd.func(us)
        osd[which(is.na(osd))] <- 1e-100
        
        #Pollen expected to go from sampled adults to hypothetical outside adults:
        ##iopol[i,j] is the total probability of pollen from the ith sampled tree's
        ##location reaching the jth distance unit outside of the plot
        iopol <- iopol.func(up)
        iopol[which(is.na(iopol))] <- 1e-100
        
        #Expected pollen arriving at sampled trees from outside:
        op    <- out.pol.func(up) 
        op[which(is.na(op))] <- 1e-100
        
        #Pollen expected to go from hypothetical outer tree to outer tree:
        otop  <- otop.func(up)
        otop[which(is.na(otop))] <- 1e-100
    } # end "if first loop"
    
    #### propose new dispersal parameters
    #current parent estimate 
    #A1 <- ped.gibbs[[d]][,1] # proposed mothers of offspring k (see noff loop below)
    #A2 <- ped.gibbs[[d]][,2] # proposed fathers of offspring k (see noff loop below)
    #faster current parent  estimate
    A1 <- ped.gibbs[,1] # proposed mothers of offspring k (see noff loop below)
    A2 <- ped.gibbs[,2] # proposed fathers of offspring k (see noff loop below)
    usstar   <- tnorm(1,los,his,us,20) #pick a new seed dispersal parameter 
    upstar   <- tnorm(1,lop,hip,up,20) #pick a new pollen dispersal parameter
    dprob.s  <- dprob.func(usstar)     #seed dispersal probs
    dprob.s[which(is.na(dprob.s))] <- 1e-100
    pprob.s  <- pprob.func(upstar)     #pollen dispersal probs
    pprob.s[which(is.na(pprob.s))] <- 1e-100
    osd.s    <- out.sd.func(usstar)    #seed to plot [row] from outside mother[col]
    osd.s[which(is.na(osd.s))]     <- 1e-100
    iopol.s  <- iopol.func(upstar)     #pollen to outside mother[col] from inside father[row] 
    iopol.s[which(is.na(iopol.s))] <- 1e-100
    op.s     <- out.pol.func(upstar)   #pollen to inside mother [row] from outside fathers [col] 
    op.s[which(is.na(op.s))]       <- 1e-100
    otop.s   <- otop.func(upstar)      #pollen to outside mother[row] from outside fathers[col]
    otop.s[which(is.na(otop.s))]   <- 1e-100
    #pollen recieved by out mother[i] from any out father:
    otopsum  <- apply(otop,1,sum)
    otopsum.s<- apply(otop.s,1,sum)
    # p        <- rep(NA,noff) 
    # pstar    <- rep(NA,noff)
    #rows = recipients, col= donors
    wmat     <- matrix(w,noff,nadult,byrow=T) # w is the weights vector
    polmat   <- matrix(w,nadult,nadult,byrow=T)
    #pld is the offspring plot size, in my case == 1m^2
    plmat    <- matrix(pld,noff,nadult) #gives a matrix where each column repeats the area of 
    #the offsprings plot sizes
    sd.pr    <- dprob*wmat*plmat 		#prob of seed dispersal from sampled adult to sampled seedling 
    sd.pr.s  <- dprob.s*wmat*plmat
    sd.pr    <- cbind(sd.pr,osd) 		#seed received from inside adults + outside adults 
    sd.pr.s  <- cbind(sd.pr.s,osd.s)
    pol.pr   <- pprob*polmat 
    pol.pr.s <- pprob.s*polmat
    pol.pr   <- cbind(pol.pr,op) 		#pollen received by adults from inside + outside adults 
    pol.pr.s <- cbind(pol.pr.s,op.s) 	#pollen received by outside adults from inside + outside adults 
    om       <- cbind(t(iopol),otop); 
    om.s     <- cbind(t(iopol.s),otop.s)
    pol.pr   <- rbind(pol.pr,om) 
    pol.pr.s <- rbind(pol.pr.s,om.s) 

    ### speed up the old for-loop
    exports = c("A1",
                "A2",
                "pol.pr",
                "pol.pr.s",
                "sd.pr",
                "sd.pr.s")
    print('exporting exports')
    clusterExport(cl,exports)
    out <- parApply(cl,loop,1,dostuff)
    p <- pstar <- tnow <- tnow.s <- tnew <- tnew.s <- pednew1 <- pednew2 <- rep(NA,noff)
    for(k in 1:noff){
        p[k]        <- out[[k]][[2]]
        pstar[k]    <- out[[k]][[3]]
        pednew1[k]	<- out[[k]][[4]]
        pednew2[k]	<- out[[k]][[5]]
        tnow[k]		<- out[[k]][[6]]
        tnow.s[k]	<- out[[k]][[7]]
        tnew[k] 	<- out[[k]][[8]]
        tnew.s[k]	<- out[[k]][[9]]
    }
    ped.new <- cbind(pednew1,pednew2)
    print('export complete')
    ### end speed up of old loop


    q1 <- sum(log(p))
    q2 <- sum(log(pstar))

    #sum over all offspring, times priors
    pnow <- q1+dnorm(up,mpp,sdpp,log=TRUE)+dnorm(us,msp,sdsp,log=TRUE)
    pnew <- q2+dnorm(upstar,mpp,sdpp,log=TRUE)+dnorm(usstar,msp,sdsp,log=TRUE)
    r    <- exp(pnew-pnow)
    z    <- runif(1,0,1) #accept or reject proposed values 
    if(z < r)
    {
        up    <- upstar 
        us    <- usstar 
        dprob <- dprob.s
        pprob <- pprob.s 
        osd   <- osd.s 
        iopol <- iopol.s 
        op    <- op.s
        otop  <- otop.s
        tnew  <- tnew.s
        tnow  <- tnow.s
    }
    x <- d
    while (nchar(x)<5)
    {
        x = sprintf("0%s",x)
    }

    saveRDS(us,fname(outdir,sprintf('usgibbs_%s.RDS',x)))
    saveRDS(up,fname(outdir,sprintf('upgibbs_%s.RDS',x)))

    ####propose pedigree 
    print('proposing pedigree')
    for(k in 1:noff) #super fast, not worth multiprocessing
    {
        #cat(sprintf("pedigree %%s\\n",k))
        #keep track of parameter values
        #probability that current or proposed parents could produce observed offspring genotype 
        #gnow <- amat[[k]][ped.gibbs[[d]][k,1],ped.gibbs[[d]][k,2]]
        #gnew <- amat[[k]][ped.new[k,1],ped.new[k,2]]
        #faster probability that current or proposed parents could produce observed offspring genotype 
        gnow <- amat[[k]][ped.gibbs[k,1],ped.gibbs[k,2]]
        gnew <- amat[[k]][ped.new[k,1],ped.new[k,2]]

        #...combined with probability of dispersal given current dispersal parameters 
        pnow <- log(gnow)+log(tnow[k])
        pnew <- log(gnew)+log(tnew[k])
        r <- exp(pnew-pnow)
        z <- runif(1,0,1)
        if(z < r)
        {#accept or reject 
            #ped.gibbs[[d+1]][k,1]<-ped.new[k,1] 
            #ped.gibbs[[d+1]][k,2]<-ped.new[k,2]
            #faster accept or reject
            ped.gibbsnext[k,1]<-ped.new[k,1] 
            ped.gibbsnext[k,2]<-ped.new[k,2]
        }
        if(z >= r)
        {
            #ped.gibbs[[d+1]][k,1]<-ped.gibbs[[d]][k,1]
            #ped.gibbs[[d+1]][k,2]<- ped.gibbs[[d]][k,2] 
            #faster
            ped.gibbsnext[k,1]<-ped.gibbs[k,1]
            ped.gibbsnext[k,2]<- ped.gibbs[k,2] 
        }
    }# end k loop over offspring 
    x <- d
    while (nchar(x)<5)
    {
        x = sprintf("0%s",x)
    }
    saveRDS(ped.gibbs,fname(outdir,sprintf("ped.gibbs_%s.RDS",x)))
    ped.gibbs <- ped.gibbsnext
#    End <- Sys.time()
#    print(d)
#    print(difftime(End,start))
#    
#    cyinfo = difftime(End,start)
#    filE = sprintf('/home/lindb/teakettle/data/gibbs/engines/rds_files/eng-%s_cyc-%s.RDS',eng,d)
#    saveRDS(cyinfo,filE)
}#end cycle
stopCluster(cl)

'''
filE = '/home/lindb/teakettle/data/gibbs/first/gibbs_20kcycles.R'
with open(filE,'w') as o:
    o.write("%s" % text)
    
text = '''#!/bin/bash
#$ -N twentyK35eng
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd
#$ -pe smp_2 35

cd %s
R --no-save < %s
''' % (op.dirname(filE),
       filE)
f = '/home/lindb/teakettle/data/gibbs/first/20kcycles_35eng.sh'
with open(f,'w') as o:
    o.write("%s" % text)

In [ ]:
!cat /home/lindb/teakettle/data/gibbs/first/20kcycles_35eng.sh

<a id="results"></a>
# combine results for analysis

In [ ]:
DIR = '/home/lindb/teakettle/data/gibbs/first/cycfiles/'
pedgibbs = [f for f in fs(DIR) if 'ped.gibbs' in f]
len(pedgibbs)

In [ ]:
pedgibbs[-1]

In [ ]:
# combine using 198 engines first, then combine 

In [ ]:
ceil = math.ceil(20000/198)
ceil

In [ ]:
blocks = []
count  = 1
xmin   = 1
for x in range(20000):
    if count == 102 or x == 19999:
        block = "%s_%s" % (str(xmin).zfill(5),str(x + 1).zfill(5))
        blocks.append(block)
        xmin  = (x + 2)
        count = 0
    count  += 1
blocks

In [ ]:
# combine ped.gibbs files into 197 blocks, then combine blocks later
rdsDIR  = '/home/lindb/teakettle/data/gibbs/first/combined/pedgibbs/rds_blocks'
rDIR = '/home/lindb/teakettle/data/gibbs/first/combined/pedgibbs/rfiles'
for d in [rdsDIR,rDIR]:
    if not op.exists(d):
        mkdir(d)
for block in blocks:
    text = '''
source('~/imports.R')

block  = as.character('%s')
splits = strsplit(block,"_")[[1]]
xmin   = as.numeric(splits[1])
xmax   = as.numeric(splits[2])

pedgibbs <- list()
for (x in xmin:xmax) {
    xchar = sprintf("%%05d",x)
    filE = sprintf('/home/lindb/teakettle/data/gibbs/first/cycfiles/ped.gibbs_%%s.RDS',xchar)
    stopifnot(file.exists(filE))
    p <- readRDS(filE)
    pedgibbs <- append(pedgibbs,list(p))
}

f = fname('%s',sprintf('ped.gibbs_%%s-%%s.RDS',splits[1],splits[2]))
saveRDS(pedgibbs,f)

''' % (block,
       rdsDIR)
    filE = op.join(rDIR,'combine_pedgibs_%s.R' % block)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
ls(rDIR)[0].split("combine_pedgibs_")[1].split('.R')[0]

In [ ]:
# write sh files to get blocks
DIR = '/home/lindb/teakettle/data/gibbs/first/combined/pedgibbs/shfiles'
if not op.exists(DIR):
    mkdir(DIR)
for f in fs(rDIR):
    num = f.split("combine_pedgibs_")[1].split('.R')[0]
    text = '''#!/bin/bash
#$ -N combped
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (rDIR,
       f)
    filE = op.join(DIR,'pedgibbs_%s.sh' % num)
    with open(filE,'w') as o:
        o.write("%s" % text)